In [ ]:
pip install opencv-python


In [ ]:
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show, show_hist
import warnings
import os
from PIL import Image
import numpy as np
import imageio
import cv2
import io
import rasterio
from rasterio.io import MemoryFile
from itertools import islice
from scipy.stats import skew
from scipy.stats import kurtosis


## Corrélation de Moran sur l'image

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt


def compute_moran_corr(plume_dir, no_plume_dir, save_plume_dir, save_no_plume_dir, window_size):

    all_files_no_plume = os.listdir(no_plume_dir)
    tif_files = [filename for filename in all_files_no_plume if filename.endswith(".tif")]

    for filename in tif_files:
        img_path = os.path.join(no_plume_dir, filename)
        img = Image.open(img_path)
        gray_image = np.array(img)

        weights = np.ones((window_size, window_size))  # Poids uniformes pour une fenêtre carrée

        # Normaliser les poids pour que leur somme soit égale à 1
        weights /= np.sum(weights)

        # Appliquer la corrélation de Moran
        moran_auto_corr = signal.convolve2d(gray_image, weights, mode='same')
        
        
        save_path = save_no_plume_dir+filename
        image_to_save = Image.fromarray(moran_auto_corr)   
        image_to_save.save(save_path)

    all_files_plume = os.listdir(plume_dir)
    tif_files = [filename for filename in all_files_plume if filename.endswith(".tif")]

    for filename in tif_files:
        img_path = os.path.join(plume_dir, filename)
        img = Image.open(img_path)
        gray_image = np.array(img)

        weights = np.ones((window_size, window_size))  # Poids uniformes pour une fenêtre carrée

        # Normaliser les poids pour que leur somme soit égale à 1
        weights /= np.sum(weights)

        # Appliquer la corrélation de Moran
        moran_auto_corr = signal.convolve2d(gray_image, weights, mode='same')
        
        save_path = save_plume_dir+filename
        image_to_save = Image.fromarray(moran_auto_corr)   
        image_to_save.save(save_path)
    
    return('done')

     



## Entropy

In [ ]:
def image_entropy(image):
    # Convertir l'image en niveaux de gris si ce n'est pas déjà le cas
    if image.ndim == 3:
        image = np.mean(image, axis=2)

    # Calculer l'histogramme des niveaux de gris
    histogram, _ = np.histogram(image, bins=256, range=(0, 256))

    # Normaliser l'histogramme pour obtenir une distribution de probabilité
    histogram = histogram / float(np.sum(histogram))

    # Calculer l'entropie en utilisant la formule de Shannon
    entropy = -np.sum(histogram * np.log2(histogram + np.finfo(float).eps))

    return entropy


# returns a list of entropy calculated for each image of images_dir, which file is in metadata.path column
def compute_entropy_list(train_data_dir, metadata_dir):
    train_data_dir = train_data_dir
    metadata = pd.read_csv(metadata_dir)

    entropy_list = []

    for filename in metadata.path:
        img_path = images_dir+filename+'.tif'
        img = Image.open(img_path)
        img_array = np.array(img)

    
        entropy_value = image_entropy(img_array)
        entropy_list.append(entropy_value)

    return(entropy_list)



## Skewness

In [ ]:
def image_skewness(image):
    # Convertir l'image en niveaux de gris si ce n'est pas déjà le cas

    # Calculer la skewness des niveaux de gris
    skewness = skew(image, axis=None)

    return skewness

def compute_skewness_list(train_data_dir, metadata_dir):
    train_data_dir = train_data_dir
    metadata = pd.read_csv(metadata_dir)

    skewness_list = []

    for filename in metadata.path:
        img_path = images_dir+filename+'.tif'
        img = Image.open(img_path)
        img_array = np.array(img)

    
        skewness_value = image_skewness(img_array)
        skewness_list.append(skewness_value)

    return(skewness_list)


## Kurtosis

In [ ]:
def image_kurtosis(image):
    # Convertir l'image en niveaux de gris si ce n'est pas déjà le cas
    if image.ndim == 3:
        image = np.mean(image, axis=2)

    # Calculer le kurtosis des niveaux de gris
    kurtosis_value = kurtosis(image, axis=None)

    return kurtosis_value

def compute_kurtosis_list(train_data_dir, metadata_dir):
    train_data_dir = train_data_dir
    metadata = pd.read_csv(metadata_dir)

    kurtosis_list = []

    for filename in metadata.path:
        img_path = images_dir+filename+'.tif'
        img = Image.open(img_path)
        img_array = np.array(img)

    
        kurtosis_value = image_kurtosis(img_array)
        kurtosis_list.append(kurtosis_value)

    return(kurtosis_list)


## Mask & plume parameters

In [ ]:
def compute_parameters(train_data_dir, metadata_dir):
    train_data_dir = train_data_dir
    metadata = pd.read_csv(metadata_dir)

    width_list = []
    height_list = []
    area_list = []
    centroid_x_list = []
    centroid_y_list = []

    for filename in metadata.path:
        img_path = images_dir+filename+'.tif'
        img = Image.open(img_path)
        image = np.array(img)

        max_value = np.max(image)

        ## create the mask regarding the max intensity of the image, the threshold
        seuil = 0.9*max_value
        resultat = np.where(image < seuil, 0, 1)
        resultat = resultat.astype(np.uint8)
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(resultat, connectivity=4)

        plus_grande_aire = 0
        label_plus_grande_aire = None

        for label in range(1, num_labels):  
            left, top, width, height, area = stats[label]
            
            if area > plus_grande_aire:
                plus_grande_aire = area
                label_plus_grande_aire = label

        # Vérifie si au moins une composante connexe a été trouvée
        if label_plus_grande_aire is not None:
            left, top, width, height, area = stats[label_plus_grande_aire]
            centroid_x, centroid_y = centroids[label_plus_grande_aire]
            width_list.append(width)
            height_list.append(height)
            area_list.append(area)
            centroid_x_list.append(centroid_x)
            centroid_y_list.append(centroid_y)

        
        else:
            print("Aucune composante connexe n'a été trouvée.")

    return(width_list, height_list, area_list, centroid_x_list, centroid_y_list)


    